In [1]:
import pandas as pd
import numpy as np

In [5]:
poss = pd.read_csv('poss_21_24.csv')
poss = poss.drop(columns=['Unnamed: 0'])

In [7]:
def calculate_player_possessions(poss):
    """
    Calculate the number of possessions each player participated in for each game.

    Parameters:
    - poss: pandas DataFrame with columns 'game_id', 'possession_id', 
            'off1', 'off2', 'off3', 'off4', 'off5',
            'def1', 'def2', 'def3', 'def4', 'def5', 'points_scored'

    Returns:
    - player_possessions: pandas DataFrame with columns 'game_id', 'playerId', 
                          'possessions_participated'
    """
    # Define offensive and defensive player columns
    off_cols = [f'off{i}' for i in range(1, 6)]
    def_cols = [f'def{i}' for i in range(1, 6)]
    
    # Select only the relevant columns
    player_columns = ['game_id', 'possession_id'] + off_cols + def_cols
    players_df = poss[player_columns].copy()
    
    # Melt the offensive players into long format
    off_melted = players_df.melt(
        id_vars=['game_id', 'possession_id'],
        value_vars=off_cols,
        var_name='position',
        value_name='playerId'
    )
    off_melted['position'] = 'offense'
    
    # Melt the defensive players into long format
    def_melted = players_df.melt(
        id_vars=['game_id', 'possession_id'],
        value_vars=def_cols,
        var_name='position',
        value_name='playerId'
    )
    def_melted['position'] = 'defense'
    
    # Combine offensive and defensive players
    all_players = pd.concat([off_melted, def_melted], ignore_index=True)
    
    # Optional: Remove any potential NaN values (if any)
    all_players = all_players.dropna(subset=['playerId'])
    
    # Now, group by 'game_id' and 'playerId' to count unique possessions
    player_possessions = all_players.groupby(['game_id', 'playerId'])['possession_id'].nunique().reset_index()
    
    # Rename the 'possession_id' column to 'possessions_participated'
    player_possessions.rename(columns={'possession_id': 'possessions_participated'}, inplace=True)
    
    # Optional: Sort the results for better readability
    player_possessions = player_possessions.sort_values(by=['game_id', 'possessions_participated'], ascending=[True, False]).reset_index(drop=True)
    
    return player_possessions



In [8]:
player_possessions_df = calculate_player_possessions(poss)

In [ ]:
player_possessions_df

,game_id,playerId,possessions_participated
0,401320565,1013,226
1,401320565,1014,226
2,401320565,3142191,226
3,401320565,3142948,226
4,401320565,3913903,226
...,...,...,...
14075,401726992,4683006,207
14076,401726992,2529622,117
14077,401726992,4398768,97
14078,401726992,2593770,84


: 

In [ ]:
bo

array([   1013, 2593770, 2566211])